In [3]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Amapá - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [17]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_AP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Amapá - Produção de Cimento (t)', axis=1)
data

,Ano,Amapá - IDH,Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Amapá - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - IDH Educacao,Amapá - Consumo de Cimento (t)
0,2014,0.716623,9.501299e+06,364732.700067,10.859668,7.367809e+06,15.577783,5.676413e+08,2.779340e+09,1.954361e+10,...,0.793503,19.134834,1010.606422,1297.941803,25.831023,23.260638,86.517021,1.031085,0.688817,227.2430
1,2015,0.720245,9.586011e+06,358753.463036,10.830618,7.371887e+06,16.255907,5.179392e+08,2.561731e+09,1.327596e+10,...,0.804619,25.746659,1010.610813,1282.207220,25.934519,23.278348,86.261290,1.002171,0.693808,198.0696
2,2016,0.719540,9.648785e+06,350101.394515,10.816782,7.379301e+06,16.439693,4.524144e+08,2.516859e+09,1.011370e+10,...,0.804039,29.398032,1010.471545,1283.328334,26.010134,23.247459,86.142602,0.972635,0.694154,166.8504
3,2017,0.718317,9.684546e+06,347692.745632,10.820598,7.391485e+06,16.090418,4.213689e+08,2.497710e+09,8.456309e+09,...,0.803042,28.039516,1010.496689,1264.603524,25.817332,23.285926,87.487872,0.956773,0.692804,135.0210
4,2018,0.717073,9.674265e+06,348410.346154,10.816868,7.388255e+06,15.812868,4.166580e+08,2.797145e+09,7.937760e+09,...,0.801501,27.404925,1010.584536,1219.360027,25.368257,23.324208,89.831025,0.921255,0.692150,123.3470
5,2019,0.714898,9.603865e+06,349107.359829,10.808345,7.367393e+06,15.509234,4.462625e+08,3.484252e+09,1.091282e+10,...,0.799455,28.162150,1010.579446,1190.802631,25.191897,23.347528,90.818816,0.904281,0.687618,120.1530
6,2020,0.712888,9.484615e+06,351761.772055,10.805525,7.338463e+06,15.329624,5.489412e+08,4.114911e+09,1.625333e+10,...,0.796860,33.084454,1010.562266,1142.884753,24.921327,23.327039,92.057082,0.878420,0.683947,133.5230
7,2021,0.711878,9.300676e+06,353376.474735,10.826249,7.304033e+06,15.114537,6.638811e+08,4.421848e+09,1.933031e+10,...,0.795211,38.839248,1010.536095,1042.318760,24.651547,23.315592,93.343196,0.852784,0.682397,157.6210
8,2022,0.714049,9.078024e+06,354954.182586,10.849033,7.260039e+06,15.201106,7.010332e+08,4.533216e+09,1.997888e+10,...,0.796776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.685615,142.7390


In [18]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Amapá - IDH,Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Taxa Selic (%),Amapá - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - IDH Educacao
0,0.716623,9.501299e+06,364732.700067,10.859668,7.367809e+06,15.577783,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,10.350442,0.793503,19.134834,1010.606422,1297.941803,25.831023,23.260638,86.517021,1.031085,0.688817
1,0.720245,9.586011e+06,358753.463036,10.830618,7.371887e+06,16.255907,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,12.454398,0.804619,25.746659,1010.610813,1282.207220,25.934519,23.278348,86.261290,1.002171,0.693808
2,0.719540,9.648785e+06,350101.394515,10.816782,7.379301e+06,16.439693,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,13.111908,0.804039,29.398032,1010.471545,1283.328334,26.010134,23.247459,86.142602,0.972635,0.694154
3,0.718317,9.684546e+06,347692.745632,10.820598,7.391485e+06,16.090418,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,9.479812,0.803042,28.039516,1010.496689,1264.603524,25.817332,23.285926,87.487872,0.956773,0.692804
4,0.717073,9.674265e+06,348410.346154,10.816868,7.388255e+06,15.812868,4.166580e+08,2.797145e+09,7.937760e+09,4.467878e+10,...,6.239828,0.801501,27.404925,1010.584536,1219.360027,25.368257,23.324208,89.831025,0.921255,0.692150
5,0.714898,9.603865e+06,349107.359829,10.808345,7.367393e+06,15.509234,4.462625e+08,3.484252e+09,1.091282e+10,4.379096e+10,...,5.780756,0.799455,28.162150,1010.579446,1190.802631,25.191897,23.347528,90.818816,0.904281,0.687618
6,0.712888,9.484615e+06,351761.772055,10.805525,7.338463e+06,15.329624,5.489412e+08,4.114911e+09,1.625333e+10,4.320994e+10,...,2.729512,0.796860,33.084454,1010.562266,1142.884753,24.921327,23.327039,92.057082,0.878420,0.683947


In [19]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    198.0696
1    166.8504
2    135.0210
3    123.3470
4    120.1530
5    133.5230
6    157.6210
7    142.7390
8         NaN
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [20]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Amapá - IDH,Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Taxa Selic (%),Amapá - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - IDH Educacao
0,-0.641459,-1.845268,1.821073,2.053877,-1.039565,-1.079253,1.779974,0.019134,2.003355,-1.880579,...,0.279224,-1.994931,-2.117789,0.891327,1.081777,0.458551,-0.861355,-0.724783,1.518000,-1.107828
1,1.362157,-0.487921,0.882374,0.308651,-0.610441,0.899475,0.870373,-0.621387,0.401153,0.111653,...,1.031513,0.952516,-0.167456,0.972569,0.672295,0.800393,-0.353644,-0.864555,0.856842,0.908967
2,0.972227,0.517919,-0.475941,-0.522510,0.169710,1.435754,-0.328802,-0.753466,-0.407217,1.154682,...,1.266612,0.798709,0.909613,-1.603893,0.701471,1.050149,-1.239144,-0.929425,0.181456,1.048854
3,0.295431,1.090922,-0.854083,-0.293281,1.451794,0.416588,-0.896969,-0.809829,-0.830898,0.955301,...,-0.032077,0.534503,0.508883,-1.138716,0.214170,0.413328,-0.136402,-0.194156,-0.181266,0.503409
4,-0.392608,0.926189,-0.741424,-0.517346,1.111920,-0.393289,-0.983185,0.071542,-0.963455,0.100493,...,-1.190564,0.125760,0.321693,0.486439,-0.963263,-1.069954,0.961015,1.086517,-0.993448,0.239033
5,-1.595749,-0.201841,-0.631998,-1.029391,-1.083417,-1.279275,-0.441390,2.094006,-0.202939,-0.441549,...,-1.354709,-0.416558,0.545057,0.392273,-1.706450,-1.652467,1.629531,1.626403,-1.381585,-1.592434


In [21]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    198.0696
1    166.8504
2    135.0210
3    123.3470
4    120.1530
5    133.5230
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [22]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Amapá - IDH,Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Taxa Selic (%),Amapá - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - IDH Educacao
6,-1.751797,-1.528418,-0.198648,-1.010922,-2.062222,-1.379599,1.16956,2.035192,0.985175,-0.705937,...,-1.662716,-0.94371,1.475696,0.068904,-1.82449,-1.698511,0.897743,1.608311,-1.464281,-1.857062


In [23]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    157.621
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [34]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500, verbose=1)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dense(2048, activation='tanh'),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(512, activation='tanh'),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [35]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [36]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1320694452, 887112600, 2266011551, 3377688589, 2444696359, 3540319763, 725027683, 2304191271, 1502491867, 994217566]


Step: 0 ___________________________________________
Epoch 1139: early stopping
val_loss: 61.607025146484375
winner_seed: 1320694452


Step: 1 ___________________________________________
Epoch 606: early stopping
val_loss: 0.013061822392046452
winner_seed: 887112600


Step: 2 ___________________________________________
Epoch 1011: early stopping
val_loss: 0.012960871681571007
winner_seed: 2266011551


Step: 3 ___________________________________________
Epoch 700: early stopping
val_loss: 0.0021659070625901222
winner_seed: 3377688589


Step: 4 ___________________________________________
Epoch 761: early stopping
val_loss: 26.799768447875977


Step: 5 ___________________________________________
Epoch 1691: early stopping
val_loss: 42.44740295410156


Step: 6 ___________________________________________


KeyboardInterrupt: 

In [37]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=3377688589)

Epoch 1/5000
1/1 [==============================] - 0s 313ms/step - loss: 22111.8145 - val_loss: 19061.8477
Epoch 2/5000
1/1 [==============================] - 0s 28ms/step - loss: 20350.1309 - val_loss: 8127.7300
Epoch 3/5000
1/1 [==============================] - 0s 28ms/step - loss: 22427.1504 - val_loss: 10787.5938
Epoch 4/5000
1/1 [==============================] - 0s 27ms/step - loss: 20250.3438 - val_loss: 19552.7539
Epoch 5/5000
1/1 [==============================] - 0s 28ms/step - loss: 17949.0020 - val_loss: 13336.6055
Epoch 6/5000
1/1 [==============================] - 0s 28ms/step - loss: 17272.8301 - val_loss: 15974.2334
Epoch 7/5000
1/1 [==============================] - 0s 28ms/step - loss: 15493.7773 - val_loss: 5395.8491
Epoch 8/5000
1/1 [==============================] - 0s 26ms/step - loss: 18148.6309 - val_loss: 7339.8770
Epoch 9/5000
1/1 [==============================] - 0s 27ms/step - loss: 15840.8604 - val_loss: 9295.4453
Epoch 10/5000
1/1 [=====================

1/1 [==============================] - 0s 29ms/step - loss: 1719.8550 - val_loss: 1517.2865
Epoch 79/5000
1/1 [==============================] - 0s 27ms/step - loss: 1670.4526 - val_loss: 1464.5092
Epoch 80/5000
1/1 [==============================] - 0s 27ms/step - loss: 1622.5297 - val_loss: 1413.4071
Epoch 81/5000
1/1 [==============================] - 0s 26ms/step - loss: 1576.0404 - val_loss: 1363.9331
Epoch 82/5000
1/1 [==============================] - 0s 26ms/step - loss: 1530.9398 - val_loss: 1316.0396
Epoch 83/5000
1/1 [==============================] - 0s 26ms/step - loss: 1487.1852 - val_loss: 1269.6766
Epoch 84/5000
1/1 [==============================] - 0s 26ms/step - loss: 1444.7339 - val_loss: 1224.7997
Epoch 85/5000
1/1 [==============================] - 0s 25ms/step - loss: 1403.5480 - val_loss: 1181.3636
Epoch 86/5000
1/1 [==============================] - 0s 35ms/step - loss: 1363.5868 - val_loss: 1139.3252
Epoch 87/5000
1/1 [==============================] - 0s 32ms

1/1 [==============================] - 0s 24ms/step - loss: 209.9497 - val_loss: 48.9263
Epoch 157/5000
1/1 [==============================] - 0s 28ms/step - loss: 205.1418 - val_loss: 45.9469
Epoch 158/5000
1/1 [==============================] - 0s 26ms/step - loss: 200.4748 - val_loss: 43.1029
Epoch 159/5000
1/1 [==============================] - 0s 26ms/step - loss: 195.9444 - val_loss: 40.3897
Epoch 160/5000
1/1 [==============================] - 0s 27ms/step - loss: 191.5466 - val_loss: 37.8025
Epoch 161/5000
1/1 [==============================] - 0s 26ms/step - loss: 187.2776 - val_loss: 35.3374
Epoch 162/5000
1/1 [==============================] - 0s 26ms/step - loss: 183.1338 - val_loss: 32.9899
Epoch 163/5000
1/1 [==============================] - 0s 28ms/step - loss: 179.1114 - val_loss: 30.7558
Epoch 164/5000
1/1 [==============================] - 0s 27ms/step - loss: 175.2065 - val_loss: 28.6312
Epoch 165/5000
1/1 [==============================] - 0s 28ms/step - loss: 171.

Epoch 236/5000
1/1 [==============================] - 0s 26ms/step - loss: 60.9973 - val_loss: 10.2963
Epoch 237/5000
1/1 [==============================] - 0s 26ms/step - loss: 60.5513 - val_loss: 10.7239
Epoch 238/5000
1/1 [==============================] - 0s 28ms/step - loss: 60.1183 - val_loss: 11.1536
Epoch 239/5000
1/1 [==============================] - 0s 26ms/step - loss: 59.6980 - val_loss: 11.5853
Epoch 240/5000
1/1 [==============================] - 0s 26ms/step - loss: 59.2900 - val_loss: 12.0186
Epoch 241/5000
1/1 [==============================] - 0s 27ms/step - loss: 58.8940 - val_loss: 12.4534
Epoch 242/5000
1/1 [==============================] - 0s 26ms/step - loss: 58.5094 - val_loss: 12.8892
Epoch 243/5000
1/1 [==============================] - 0s 26ms/step - loss: 58.1362 - val_loss: 13.3260
Epoch 244/5000
1/1 [==============================] - 0s 27ms/step - loss: 57.7739 - val_loss: 13.7635
Epoch 245/5000
1/1 [==============================] - 0s 25ms/step - loss

1/1 [==============================] - 0s 26ms/step - loss: 47.2198 - val_loss: 39.6608
Epoch 316/5000
1/1 [==============================] - 0s 28ms/step - loss: 47.1771 - val_loss: 39.9178
Epoch 317/5000
1/1 [==============================] - 0s 27ms/step - loss: 47.1358 - val_loss: 40.1720
Epoch 318/5000
1/1 [==============================] - 0s 27ms/step - loss: 47.0956 - val_loss: 40.4234
Epoch 319/5000
1/1 [==============================] - 0s 27ms/step - loss: 47.0566 - val_loss: 40.6716
Epoch 320/5000
1/1 [==============================] - 0s 27ms/step - loss: 47.0187 - val_loss: 40.9172
Epoch 321/5000
1/1 [==============================] - 0s 27ms/step - loss: 46.9820 - val_loss: 41.1596
Epoch 322/5000
1/1 [==============================] - 0s 26ms/step - loss: 46.9463 - val_loss: 41.3992
Epoch 323/5000
1/1 [==============================] - 0s 29ms/step - loss: 46.9117 - val_loss: 41.6361
Epoch 324/5000
1/1 [==============================] - 0s 27ms/step - loss: 46.8781 - val

1/1 [==============================] - 0s 27ms/step - loss: 45.9029 - val_loss: 52.6006
Epoch 395/5000
1/1 [==============================] - 0s 26ms/step - loss: 45.8989 - val_loss: 52.6922
Epoch 396/5000
1/1 [==============================] - 0s 26ms/step - loss: 45.8949 - val_loss: 52.7822
Epoch 397/5000
1/1 [==============================] - 0s 26ms/step - loss: 45.8911 - val_loss: 52.8712
Epoch 398/5000
1/1 [==============================] - 0s 27ms/step - loss: 45.8873 - val_loss: 52.9588
Epoch 399/5000
1/1 [==============================] - 0s 26ms/step - loss: 45.8837 - val_loss: 53.0453
Epoch 400/5000
1/1 [==============================] - 0s 27ms/step - loss: 45.8802 - val_loss: 53.1307
Epoch 401/5000
1/1 [==============================] - 0s 28ms/step - loss: 45.8768 - val_loss: 53.2148
Epoch 402/5000
1/1 [==============================] - 0s 26ms/step - loss: 45.8735 - val_loss: 53.2976
Epoch 403/5000
1/1 [==============================] - 0s 26ms/step - loss: 45.8703 - val

1/1 [==============================] - 0s 27ms/step - loss: 45.7771 - val_loss: 56.9712
Epoch 474/5000
1/1 [==============================] - 0s 27ms/step - loss: 45.7766 - val_loss: 57.0007
Epoch 475/5000
1/1 [==============================] - 0s 26ms/step - loss: 45.7763 - val_loss: 57.0295
Epoch 476/5000
1/1 [==============================] - 0s 26ms/step - loss: 45.7759 - val_loss: 57.0581
Epoch 477/5000
1/1 [==============================] - 0s 27ms/step - loss: 45.7756 - val_loss: 57.0862
Epoch 478/5000
1/1 [==============================] - 0s 27ms/step - loss: 45.7752 - val_loss: 57.1141
Epoch 479/5000
1/1 [==============================] - 0s 29ms/step - loss: 45.7749 - val_loss: 57.1413
Epoch 480/5000
1/1 [==============================] - 0s 26ms/step - loss: 45.7745 - val_loss: 57.1681
Epoch 481/5000
1/1 [==============================] - 0s 28ms/step - loss: 45.7742 - val_loss: 57.1949
Epoch 482/5000
1/1 [==============================] - 0s 27ms/step - loss: 45.7739 - val

1/1 [==============================] - 0s 27ms/step - loss: 45.7650 - val_loss: 58.3575
Epoch 553/5000
1/1 [==============================] - 0s 25ms/step - loss: 45.7649 - val_loss: 58.3666
Epoch 554/5000
1/1 [==============================] - 0s 32ms/step - loss: 45.7649 - val_loss: 58.3754
Epoch 555/5000
1/1 [==============================] - 0s 29ms/step - loss: 45.7649 - val_loss: 58.3845
Epoch 556/5000
1/1 [==============================] - 0s 26ms/step - loss: 45.7648 - val_loss: 58.3931
Epoch 557/5000
1/1 [==============================] - 0s 26ms/step - loss: 45.7648 - val_loss: 58.4020
Epoch 558/5000
1/1 [==============================] - 0s 27ms/step - loss: 45.7648 - val_loss: 58.4104
Epoch 559/5000
1/1 [==============================] - 0s 28ms/step - loss: 45.7648 - val_loss: 58.4190
Epoch 560/5000
1/1 [==============================] - 0s 26ms/step - loss: 45.7648 - val_loss: 58.4272
Epoch 561/5000
1/1 [==============================] - 0s 27ms/step - loss: 45.7647 - val

1/1 [==============================] - 0s 27ms/step - loss: 45.7639 - val_loss: 58.7891
Epoch 632/5000
1/1 [==============================] - 0s 26ms/step - loss: 45.7638 - val_loss: 58.7919
Epoch 633/5000
1/1 [==============================] - 0s 28ms/step - loss: 45.7638 - val_loss: 58.7949
Epoch 634/5000
1/1 [==============================] - 0s 28ms/step - loss: 45.7638 - val_loss: 58.7975
Epoch 635/5000
1/1 [==============================] - 0s 26ms/step - loss: 45.7638 - val_loss: 58.8003
Epoch 636/5000
1/1 [==============================] - 0s 26ms/step - loss: 45.7639 - val_loss: 58.8029
Epoch 637/5000
1/1 [==============================] - 0s 26ms/step - loss: 45.7638 - val_loss: 58.8055
Epoch 638/5000
1/1 [==============================] - 0s 27ms/step - loss: 45.7638 - val_loss: 58.8083
Epoch 639/5000
1/1 [==============================] - 0s 25ms/step - loss: 45.7638 - val_loss: 58.8108
Epoch 640/5000
1/1 [==============================] - 0s 26ms/step - loss: 45.7638 - val

In [38]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 35ms/step
[157.621](test_target) - [[141.1985]](prediction) = 16.422498413085947


In [39]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [44]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, 3377688589)
predictions

train_input:


,Amapá - IDH,Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Taxa Selic (%),Amapá - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - IDH Educacao
0,-1.39019,-1.280755,1.143822,1.341727,-1.091023,-1.384937,1.165029,1.396038,1.335266,-1.330895,...,-1.376995,-1.412693,-1.324267,0.672867,1.411327,-1.283121,-0.119371,1.344479,1.220376,-1.411836
1,0.91988,0.121003,0.148334,-0.283789,-0.233752,0.444553,0.111769,-0.502296,-0.264151,0.251276,...,0.409368,0.763117,0.232321,0.740803,-0.783869,0.126584,1.280059,-0.292393,0.008692,0.634935
2,0.47031,1.159752,-1.292156,-1.057938,1.324775,0.940384,-1.276798,-0.893743,-1.071116,1.079619,...,0.967627,0.649576,1.091945,-1.413670,-0.627458,1.156537,-1.160689,-1.052085,-1.229068,0.776901


train_target:


,Amapá - Consumo de Cimento (t)
0,198.0696
1,166.8504
2,135.0210


test_input:


...

,Amapá - IDH,Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Taxa Selic (%),Amapá - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - IDH Educacao
6,-1.751797,-1.528418,-0.198648,-1.010922,-2.062222,-1.379599,1.16956,2.035192,0.985175,-0.705937,...,-1.662716,-0.94371,1.475696,0.068904,-1.82449,-1.698511,0.897743,1.608311,-1.464281,-1.857062


test_target:


,Amapá - Consumo de Cimento (t)
6,157.621


Epoch 700: early stopping
1/1 [==============================] - 0s 32ms/step
Error: 16.422498413085947




[135.02052307128906, 160.4148712158203, 159.10552978515625, 141.19850158691406]

In [45]:
display(targets)
display(predictions)

[123.347, 120.153, 133.523, 157.621]

[135.02052307128906, 160.4148712158203, 159.10552978515625, 141.19850158691406]

In [46]:
mae = mean_absolute_error(predictions, targets)
mae

23.485105621337894

In [47]:
porcentage = mae/np.mean(targets)
porcentage

0.175706493452375